In [ ]:
from tdmclient import ClientAsync
from vision.ComputerVision import Vision
import time
import cv2
import motion_functions
import matplotlib.pyplot as plt
import filtering
import numpy as np

vis = Vision()
client = ClientAsync()

node = await client.wait_for_node()
await node.lock()

last_image_time = time.time()
dt = 0.05
path = []

state = 0 # FSM state
target_node = 0 # Initial node
goal_reached = False

while True:
    while state == 0: # Goal, robot and path aquisition state
        while time.time()-last_image_time < dt: # Aquire a new image every dt seconds
            continue
        plt.imshow(vis.copy)
        plt.show()
        
        vis.update()
        last_image_time = time.time()
        path = vis.shortest_path
        if len(path)>=1:
            state = 1
            rob_pos = [vis.robot.x, vis.robot.y, vis.robot.angle]

    while state == 1: # Drive towards goal state
        while time.time()-last_image_time < dt: # Aquire a new image every dt seconds
            continue
        vis.show()
        prev_z = np.array([vis.robot.x, vis.robot.y, vis.robot.angle])
        vis.update()
        
        # Setting up and running the Kalman Filter
        await client.sleep(0.01)
        await node.wait_for_variables()
        speed_left = node.v.motor.left.speed
        speed_right = node.v.motor.right.speed
        prev_angle = rob_pos[2]
        rob_pos = filtering.run_filter(speed_right, speed_left, prev_angle, vis, prev_z)
        
        # Checking if we arrived at the next node
        arrived_node = motion_functions.close_coordinates(rob_pos[0], rob_pos[1], path[target_node][0], path[target_node][1])
        if (arrived_node and (target_node) < len(path)):
            target_node += 1
        else:
            state = 5 # Stopping state
            break # Exiting this state
        
        # Rotating and driving towards the next node
        _,angle = motion_functions.compute_movement([rob_pos[0], rob_pos[1]], vis.goal.x, vis.goal.y, rob_pos[2])
        if (abs(angle)>10):
            await motion_functions.rotate(angle)
        else:
            await motion_functions.drive()
    
    while state == 5: # State to stop and exit the program
        await motion_functions.stop()
        goal_reached = True
        break
    
    if goal_reached:
        break

del vis
await node.unlock()